# Solves perfect information sequential games

In [1]:
import numpy as np
import pdb

In [8]:
class Node:
    nodeid = 0
    "Node in a tree" 
    def __init__(self, parent):
        self.parent = parent
        self.depth = (0 if self.is_root() else parent.depth + 1)
        self.children = []
        self.nodeid = Node.nodeid
        Node.nodeid += 1
        
    def is_root(self):
        return self.parent == self
    
    def reset_nodeid():
        Node.nodeid = 0

class ActionNode(Node):
    "Type of node where a decision must be taken"
    def __init__(self, parent, player):
        super().__init__(parent)
        self.player = player
        self.decision = np.nan
        self.value = (np.nan, np.nan)
    
    def reproduce_actions(self, lam, minactions, maxactions):
        nchildren = np.clip(np.random.poisson(lam), minactions, maxactions)
        next_player = (self.player + 1) % 2
        self.children = [ActionNode(self, next_player) for _ in range(nchildren)]
        
    def reproduce_payoffs(self, lam, minpayoffs, maxpayoffs):
        nchild = np.clip(np.random.poisson(lam), minpayoffs, maxpayoffs) # at least one
        payoffs0 = [np.random.choice([-1, 0, 1]) for _ in range(nchild)]
        self.children = [PayOffNode(self, (-v, v)) for v in payoffs0]
        
    def __repr__(self):
        decision_str C= "" if np.isnan(self.decision) else "selected: {}".format(self.decision)
        value_str = "" if np.isnan(self.value[0]) else "value: ({}, {})".format(self.value[0], self.value[1])
        return "Action(id: {}, player: {}, {}, {})".\
            format(self.nodeid, self.player, decision_str, value_str)
    
class GameRootNode(ActionNode):
    "Regular action node but self refential"
    def __init__(self):
        parent = self
        player = 0
        super().__init__(parent, player)    
    
class PayOffNode(Node):
    "A node with a payoff value"
    def __init__(self, parent, payoff):
        assert len(payoff) == 2
        super().__init__(parent)
        self.payoff = payoff
        parent.children = [self]
        self.selected = np.nan
        
    def __repr__(self):
        return "PayOff(id: {}, value: ({:d}, {:d}))".format(self.nodeid, *self.payoff)

class TwoPlayerGame:
    def __init__(self, 
                 root=GameRootNode(), 
                 lam=2., maxdepth=10,
                 generate=True,
                 minactions=0, 
                 maxactions=99, 
                 minpayoffs=2,
                 maxpayoffs=99):
        self.root = root
        self.lam = lam
        self.minactions=minactions
        self.maxactions=maxactions
        self.minpayoffs=minpayoffs
        self.maxpayoffs=maxpayoffs
        self.maxdepth = maxdepth
        self.solved = False
        self.num_nodes = 1
        
        if generate:
            self.generate()
      
    def generate(self):      
        "Randomly reproduces nodes until maxdepth or no descendents found"
        # while a branch has a non-payoff node
        Node.reset_nodeid()
        nonpayoff = [self.root]
        while len(nonpayoff) > 0:
            # take one nonterminal node and reproduce
            node = nonpayoff.pop()
            if node.depth < self.maxdepth - 1:
                node.reproduce_actions(self.lam, minactions=self.minactions, maxactions=self.maxactions)
                if len(node.children) > 0:     
                    for child in node.children:
                        nonpayoff.append(child)
                        self.num_nodes += 1
                else:
                    # spawn a generation of payoffs
                    node.reproduce_payoffs(self.lam, self.minpayoffs, self.maxpayoffs)
                    self.num_nodes += len(node.children)
    
    def branch(self, node):
        "returns a pointer to the same fields of the tree but different root"
        return TwoPlayerGame(root=node, generate=False)
    
    def solve(self):
        "uses backward induction (dynamic programming) to find a Nash equlibrium"
        node = self.root      
        if isinstance(node, ActionNode):
            player = node.player
            children = node.children
            child_value = []
            for i, child in enumerate(children):
                if isinstance(child, PayOffNode):
                    child_value.append(child.payoff)
                else:
                    child_subgame = self.branch(child)
                    child_subgame.solve()
                    subgame_value = child_subgame.root.value
                    child_value.append(subgame_value)
    
            node.decision = np.argmax([x[player] for x in child_value])
            node.value = child_value[node.decision]
        self.solved = True
        
    def print_solution_path(self):
        "Prints selected nodes only"
        assert self.solved
        node = self.root
        s = ('  ' * node.depth) + str(node) + '\n'
        while not isinstance(node, PayOffNode):
            node = node.children[node.decision]
            s += (':-' * node.depth) + str(node) + '\n'
        if node.payoff[0] > node.payoff[1]:
            s += "Player 0 wins"
        elif node.payoff[1] > node.payoff[0]:
            s += "Player 1 wins"
        else:
            s += "Player 0 and 1 draw"
        print(s)

    def __repr__(self):
        return "Tree with {:d} nodes".format(self.num_nodes) 

    def __str__(self):
        "Depth-First printing"
        node = self.root
        strout = (':-' * node.depth) + str(node) + '\n'
        if isinstance(node, PayOffNode):
            return strout
        else:
            children = node.children
            for child in children:
                strout += self.branch(child).__str__()
            return strout
        
    def valuemap(self, playerid):
        # we do BFS for saving in a dict all values
        assert self.solved
        valuemap = dict()
        curr=self.root
        to_visit=curr.children.copy()
        while len(to_visit) > 0:
            curr = to_visit.pop()
            if isinstance(curr, PayOffNode):
                valuemap[curr.nodeid]=curr.payoff[playerid]
            else:
                valuemap[curr.nodeid]=curr.value[playerid]
                for x in curr.children:
                    to_visit.append(x)
        return valuemap

In [11]:
np.random.seed(1244)
game = TwoPlayerGame(lam = 1.1, minpayoffs=2, maxdepth = 6)
print(game)

Action(id: 0, player: 0, selected: 0, value: (0, 0))
:-Action(id: 926, player: 1, , )
:-:-Action(id: 940, player: 0, , )
:-:-:-Action(id: 957, player: 1, , )
:-:-:-:-Action(id: 974, player: 0, , )
:-:-:-:-:-Action(id: 985, player: 1, , )
:-:-:-:-:-:-PayOff(id: 987, value: (-1, 1))
:-:-:-:-:-:-PayOff(id: 988, value: (0, 0))
:-:-:-:-Action(id: 975, player: 0, , )
:-:-:-:-:-Action(id: 976, player: 1, , )
:-:-:-:-:-:-PayOff(id: 983, value: (1, -1))
:-:-:-:-:-:-PayOff(id: 984, value: (-1, 1))
:-:-:-:-:-Action(id: 977, player: 1, , )
:-:-:-:-:-:-PayOff(id: 979, value: (1, -1))
:-:-:-:-:-:-PayOff(id: 980, value: (0, 0))
:-:-:-:-:-:-PayOff(id: 981, value: (-1, 1))
:-:-:-Action(id: 958, player: 1, , )
:-:-:-:-Action(id: 967, player: 0, , )
:-:-:-:-:-PayOff(id: 971, value: (-1, 1))
:-:-:-:-:-PayOff(id: 972, value: (1, -1))
:-:-:-:-:-PayOff(id: 973, value: (-1, 1))
:-:-:-:-Action(id: 968, player: 0, , )
:-:-:-:-:-PayOff(id: 969, value: (-1, 1))
:-:-:-:-:-PayOff(id: 970, value: (1, -1))
:-:-:-Acti

In [12]:
game.solve()
game.print_solution_path()

Action(id: 0, player: 0, selected: 1, value: (1, -1))
:-Action(id: 927, player: 1, selected: 0, value: (1, -1))
:-:-Action(id: 933, player: 0, selected: 3, value: (1, -1))
:-:-:-PayOff(id: 937, value: (1, -1))
Player 0 wins


In [13]:
print(game)

Action(id: 0, player: 0, selected: 1, value: (1, -1))
:-Action(id: 926, player: 1, selected: 1, value: (0, 0))
:-:-Action(id: 940, player: 0, selected: 1, value: (1, -1))
:-:-:-Action(id: 957, player: 1, selected: 0, value: (-1, 1))
:-:-:-:-Action(id: 974, player: 0, selected: 0, value: (-1, 1))
:-:-:-:-:-Action(id: 985, player: 1, selected: 0, value: (-1, 1))
:-:-:-:-:-:-PayOff(id: 987, value: (-1, 1))
:-:-:-:-:-:-PayOff(id: 988, value: (0, 0))
:-:-:-:-Action(id: 975, player: 0, selected: 0, value: (-1, 1))
:-:-:-:-:-Action(id: 976, player: 1, selected: 1, value: (-1, 1))
:-:-:-:-:-:-PayOff(id: 983, value: (1, -1))
:-:-:-:-:-:-PayOff(id: 984, value: (-1, 1))
:-:-:-:-:-Action(id: 977, player: 1, selected: 2, value: (-1, 1))
:-:-:-:-:-:-PayOff(id: 979, value: (1, -1))
:-:-:-:-:-:-PayOff(id: 980, value: (0, 0))
:-:-:-:-:-:-PayOff(id: 981, value: (-1, 1))
:-:-:-Action(id: 958, player: 1, selected: 0, value: (1, -1))
:-:-:-:-Action(id: 967, player: 0, selected: 1, value: (1, -1))
:-:-:-:-:

In [22]:
np.random.seed(9809)
Node.reset_nodeid()
game2 = TwoPlayerGame(minactions=2, maxactions=2, maxdepth=18)
game2

Tree with 786539 nodes

In [23]:
game2.solve()
game2.print_solution_path()

Action(id: 0, player: 0, selected: 0, value: (-1, 1))
:-Action(id: 0, player: 1, selected: 0, value: (-1, 1))
:-:-Action(id: 523682, player: 0, selected: 0, value: (-1, 1))
:-:-:-Action(id: 786119, player: 1, selected: 0, value: (-1, 1))
:-:-:-:-Action(id: 917183, player: 0, selected: 0, value: (-1, 1))
:-:-:-:-:-Action(id: 982917, player: 1, selected: 0, value: (-1, 1))
:-:-:-:-:-:-Action(id: 1015805, player: 0, selected: 0, value: (-1, 1))
:-:-:-:-:-:-:-Action(id: 1032287, player: 1, selected: 0, value: (-1, 1))
:-:-:-:-:-:-:-:-Action(id: 1040505, player: 0, selected: 0, value: (-1, 1))
:-:-:-:-:-:-:-:-:-Action(id: 1044573, player: 1, selected: 0, value: (-1, 1))
:-:-:-:-:-:-:-:-:-:-Action(id: 1046630, player: 0, selected: 0, value: (-1, 1))
:-:-:-:-:-:-:-:-:-:-:-Action(id: 1047653, player: 1, selected: 1, value: (-1, 1))
:-:-:-:-:-:-:-:-:-:-:-:-Action(id: 1048156, player: 0, selected: 0, value: (-1, 1))
:-:-:-:-:-:-:-:-:-:-:-:-:-Action(id: 1048157, player: 1, selected: 0, value: (-1